# Introduction

This notebook shows how to configure GPTCache to use weaviate as the vector store

# Imports

In [1]:
from gptcache import cache
from gptcache.manager import get_data_manager, CacheBase, VectorBase
from gptcache.similarity_evaluation.distance import SearchDistanceEvaluation
from gptcache.embedding import OpenAI
import weaviate
import os
import sqlite3
from gptcache.adapter import openai
import timeit

# Setup

Use openai to do the embedding:

In [2]:
openai_embedding_fn = OpenAI().to_embeddings

In [3]:
openai_embedding_fn("hello world")

array([-0.01494597,  0.0014304 , -0.01848823, ..., -0.03047134,
       -0.00347313, -0.01090331], dtype=float32)

Use sqlite to cache the request/response:

In [4]:
cache_base = CacheBase("sqlite")

/workspaces/howto-gptcache-weaviate/.venv/lib/python3.11/site-packages/gptcache/manager/scalar_data/sql_storage.py:45: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  DynamicBase = declarative_base(class_registry={})  # pylint: disable=C0103


What's in the sqlite db now?

In [5]:
def dump_cache_content():
    cursor = sqlite3.connect("sqlite.db").cursor()
    tables = cursor.execute('SELECT * FROM sqlite_master WHERE type="table"').fetchall()

    for table in tables:
        # Print the table name as a delimiter
        print(f"Results for table {table[1]}:")
        print("------------------------")

        # Execute a SELECT * query for the table
        cursor.execute(f"SELECT * FROM {table[1]}")
        results = cursor.fetchall()

        # Print the results
        for row in results:
            print(row)

        # Print a blank line to separate the output for each table
        print()

In [6]:
dump_cache_content()

Results for table gptcache_question:
------------------------

Results for table gptcache_answer:
------------------------

Results for table gptcache_question_dep:
------------------------

Results for table gptcache_session:
------------------------

Results for table gptcache_report:
------------------------



Use weaviate to store the vector:

In [7]:
url = os.getenv("WEAVIATE_URL")
api_key = os.getenv("WEAVIATE_API_KEY")
auth_config = weaviate.AuthApiKey(api_key=api_key)
vector_base = VectorBase("weaviate", url=url, auth_client_secret=auth_config)

start to install package: weaviate-client
successfully installed package: weaviate-client


Create a weaviate client to query the db outside gptcache:

In [8]:
weaviate_client = weaviate.Client(url=url, auth_client_secret=auth_config)

Check that the weaviate db is empty:

In [9]:
weaviate_class_name = "GPTCache"
weaviate_client.schema.get(class_name=weaviate_class_name)

{'class': 'GPTCache',
 'description': 'LLM response cache',
 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
  'cleanupIntervalSeconds': 60,
  'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
 'multiTenancyConfig': {'enabled': False},
 'properties': [{'dataType': ['int'],
   'description': 'The data-id generated by GPTCache for vectors.',
   'indexFilterable': True,
   'indexSearchable': False,
   'name': 'data_id'}],
 'replicationConfig': {'factor': 1},
 'shardingConfig': {'virtualPerPhysical': 128,
  'desiredCount': 1,
  'actualCount': 1,
  'desiredVirtualCount': 128,
  'actualVirtualCount': 128,
  'key': '_id',
  'strategy': 'hash',
  'function': 'murmur3'},
 'vectorIndexConfig': {'skip': False,
  'cleanupIntervalSeconds': 300,
  'maxConnections': 64,
  'efConstruction': 128,
  'ef': -1,
  'dynamicEfMin': 100,
  'dynamicEfMax': 500,
  'dynamicEfFactor': 8,
  'vectorCacheMaxObjects': 1000000000000,
  'flatSearchCutoff': 40000,
  'distance': 'cosine',
  'pq

In [10]:
weaviate_client.data_object.get(class_name=weaviate_class_name)

{'deprecations': [], 'objects': []}

Back to GPTCache stuff, we init the cache:

In [11]:
data_manager = get_data_manager(cache_base, vector_base)

cache.init(
    embedding_func=openai_embedding_fn,
    data_manager=data_manager,
    similarity_evaluation=SearchDistanceEvaluation(max_distance=1),
)

cache.set_openai_key()

Note:

In `SearchDistanceEvaluation(max_distance=1)`, we set `max_distance=1` to make the similarity threshold calculation "work" using this evaluation metric and cosine similarity (weaviate's default similarity metric).

References:

1. [calculating rank threshold](https://github.com/zilliztech/GPTCache/blob/03a059704443961ae5b6ca243e3edc2dc15aeb2a/gptcache/adapter/adapter.py#L98C1-L107C10)

2. [applying the rank threshold](https://github.com/zilliztech/GPTCache/blob/03a059704443961ae5b6ca243e3edc2dc15aeb2a/gptcache/adapter/adapter.py#L158C1-L176C18)

# Usage

Create some functions to calculate the time it takes to query the LLM:

In [12]:
def timeit_decorator(func):
    def wrapper(*args, **kwargs):
        # Time the execution of the function
        start_time = timeit.default_timer()
        result = func(*args, **kwargs)
        end_time = timeit.default_timer()

        # Print the time taken
        print(f"Time taken to run {func.__name__}: {end_time - start_time:.2f} seconds")

        return result

    return wrapper

In [13]:
# Define a function to get a response from OpenAI's GPT-3 API
@timeit_decorator
def get_openai_response(question):
    # Call the OpenAI API to get a response
    result = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": question}],
    )
    # Extract the response from the API result
    response = result["choices"][0]["message"]["content"]

    # Return the response
    return response

Let's ask the llm "Who is Barrack Obama":

In [14]:
question = "Who is Barrack Obama?"

get_openai_response(question)

Time taken to run get_openai_response: 4.12 seconds


'Barack Obama is an American politician and attorney who served as the 44th President of the United States from 2009 to 2017. He is a member of the Democratic Party and is known for being the first African American to hold the office of President. During his tenure, Obama implemented significant policy changes, including the Affordable Care Act (also known as Obamacare) and the economic stimulus package to recover from the 2008 financial crisis. He is considered a charismatic and influential figure in American politics.'

Let's ask it again and note how long the call took:

In [15]:
get_openai_response(question)

Time taken to run get_openai_response: 0.56 seconds


'Barack Obama is an American politician and attorney who served as the 44th President of the United States from 2009 to 2017. He is a member of the Democratic Party and is known for being the first African American to hold the office of President. During his tenure, Obama implemented significant policy changes, including the Affordable Care Act (also known as Obamacare) and the economic stimulus package to recover from the 2008 financial crisis. He is considered a charismatic and influential figure in American politics.'

Let's rephrase the question:

In [16]:
rephrase_question = "Tell me more about Barrak Obama"
get_openai_response(rephrase_question)

Time taken to run get_openai_response: 0.46 seconds


'Barack Obama is an American politician and attorney who served as the 44th President of the United States from 2009 to 2017. He is a member of the Democratic Party and is known for being the first African American to hold the office of President. During his tenure, Obama implemented significant policy changes, including the Affordable Care Act (also known as Obamacare) and the economic stimulus package to recover from the 2008 financial crisis. He is considered a charismatic and influential figure in American politics.'

Look at the content of the dbs now:

In [17]:
dump_cache_content()

Results for table gptcache_question:
------------------------
(1, 'Who is Barrack Obama?', '2023-07-27 18:35:38.343857', '2023-07-27 18:35:39.434751', b'\xaa\x8d=\xbc\xe0O\x04\xbd\xd4\xe9Y\xbc\xf3\xd8\x93\xbc(\xf7d\xbcP\x91\xb6\xbbR6\xb5\xbb\x0e\xf6\x1d;]\xf8\xe9\xbcx+\x88\xbb$\xad\xe7<\xbdk\x1f\xbb\x1f\x13>;d\xc5\xf3:\xa1p\x87<\xf1l\xa4\xbb\x1c\xe5\x03=\x1d\xc3\x91\xbc\xf5\xb6\xa1<\xbdk\x9f\xbcK\x14Y\xbc\xce\x16\xa1\xbc\xecD\x19<\x03\xdf\x16<\x06\xf0\x84\xbbD\x80^;/\xc4\xee<\x1d\xfc\xa0\xbc\x9a\x15\x1c<\xe3}>\xbdY\x91\xa0<\xab2\xbc\xbc\n:\x82\xbc\xe2\xbc|\xbc4\xcf\xad\xbc\xd7\x88)\xbc\x83\x98\xea\xbb\xecD\x19\xbc+$\x96<\x05\x84\x159\xbb\xe3\xec<\x9aN\xab;\x8c{\x88\xba\xd4"\xe9\xbb\x18\r\xa5\xbc\x00\x078\xbc\x0f\x7fY\xba\xf3\x9f\x04\xbd\xa6\xb5\xde\xbcj\x03\x93<\x03\xa6\x07=\xb4\xa4\xc4<pd\xad\xbb\xcd\xc7}\xbc/\xe01\xbcu\xe1\x8a\xbb\xa3\x15\x06\xbd\xf3\xd8\x13\xb9+zq\xbb\xdb}\xd4\xba=zE<\xb4\xc0\x07<M\xd5\x1a\xbcA\xa8\xff;\x85\x92\xbb;|\xcb`\xbc\xcdU_\xbc+\x964=GX=\xbc\x87\xe2g<\xc7\x8

In [18]:
# weaviate_client.data_object.get(class_name=weaviate_class_name, with_vector=True)
weaviate_client.data_object.get(class_name=weaviate_class_name)

{'deprecations': [],
 'objects': [{'class': 'GPTCache',
   'creationTimeUnix': 1690482938488,
   'id': '9e386de1-2f45-439c-8e26-b18b9d377bd1',
   'lastUpdateTimeUnix': 1690482938488,
   'properties': {'data_id': 1},
   'vectorWeights': None}],
 'totalResults': 1}

Now look at how semantic caching can fail:

In [19]:
new_question = "Who is Joe Biden?"

get_openai_response(new_question)

Time taken to run get_openai_response: 0.28 seconds


'Barack Obama is an American politician and attorney who served as the 44th President of the United States from 2009 to 2017. He is a member of the Democratic Party and is known for being the first African American to hold the office of President. During his tenure, Obama implemented significant policy changes, including the Affordable Care Act (also known as Obamacare) and the economic stimulus package to recover from the 2008 financial crisis. He is considered a charismatic and influential figure in American politics.'

In [20]:
another_new_question = "Who is Taylor Swift?"

get_openai_response(another_new_question)

Time taken to run get_openai_response: 0.42 seconds


'Barack Obama is an American politician and attorney who served as the 44th President of the United States from 2009 to 2017. He is a member of the Democratic Party and is known for being the first African American to hold the office of President. During his tenure, Obama implemented significant policy changes, including the Affordable Care Act (also known as Obamacare) and the economic stimulus package to recover from the 2008 financial crisis. He is considered a charismatic and influential figure in American politics.'

In [21]:
get_openai_response("Who is Miley Cyrus?")

Time taken to run get_openai_response: 0.27 seconds


'Barack Obama is an American politician and attorney who served as the 44th President of the United States from 2009 to 2017. He is a member of the Democratic Party and is known for being the first African American to hold the office of President. During his tenure, Obama implemented significant policy changes, including the Affordable Care Act (also known as Obamacare) and the economic stimulus package to recover from the 2008 financial crisis. He is considered a charismatic and influential figure in American politics.'

In [22]:
get_openai_response("Who is Antman?")

Time taken to run get_openai_response: 4.88 seconds


'Ant-Man is a fictional superhero character primarily seen in Marvel Comics. The character was created by writer Stan Lee, Larry Lieber, and artist Jack Kirby, and first appeared in Tales to Astonish #27 in January 1962. The original Ant-Man was scientist Dr. Henry Pym, who developed a substance that allowed him to change his size and communicate with ants. Pym later passed on the mantle to other characters, including Scott Lang and Eric O\'Grady, who became Ant-Man in different storylines. The character has also appeared in various Marvel films, most notably in the standalone "Ant-Man" movies released in 2015 and 2018, portrayed by actor Paul Rudd.'

In [23]:
get_openai_response("Who is Spiderman?")

Time taken to run get_openai_response: 0.49 seconds


'Ant-Man is a fictional superhero character primarily seen in Marvel Comics. The character was created by writer Stan Lee, Larry Lieber, and artist Jack Kirby, and first appeared in Tales to Astonish #27 in January 1962. The original Ant-Man was scientist Dr. Henry Pym, who developed a substance that allowed him to change his size and communicate with ants. Pym later passed on the mantle to other characters, including Scott Lang and Eric O\'Grady, who became Ant-Man in different storylines. The character has also appeared in various Marvel films, most notably in the standalone "Ant-Man" movies released in 2015 and 2018, portrayed by actor Paul Rudd.'

Questions:

1. Why does Barrak Obama, Joe Biden, Taylor Swift, Miley Cyrus semantically similar?
2. How to tweak semantic caching so that "Who is Barrack Obama" and "Who is Joe Biden" are semantically distinct? Do we need more sophisticated distance evaluation metric?
3. In what use cases does semantic caching make sense?